In [162]:
##Swap Pricer
import numpy as np
from itertools import product
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
from tkinter import*

In [ ]:
#Terms:
#Fixed Leg: semiannual pmts, 30/360 daycount, NYC Holidays
#Float Leg: quarterly pmts, ACT/360 daycount, 3ML w 2 day lookback,NYC & Lon Holidays
#Discount w the OIS Curve

In [163]:
def bdayconvert(start,end,dates):
    i = 0
    while i in range(len(dates)):
        if dates[i] not in pd.bdate_range(start,end):
            dates[i] = dates[i]+relativedelta(days=+1)
            i = 0
        else:
            i+=1
    return dates

In [164]:
def pmtdates(start,end):
    sdate = datetime.datetime.strptime(start, '%Y-%m-%d')
    edate = datetime.datetime.strptime(end, '%Y-%m-%d')
    delta = edate-sdate
#    fixedpmtdates = [sdate+relativedelta(months=+6)]
    nyears = int(delta.days/365)
#    i = 1
#    while i < nyears*2:
#        fixedpmtdates.append(fixedpmtdates[i-1]+relativedelta(months=+6))
#        i+=1
    floatpmtdates = [sdate+relativedelta(months=+3)]
    i = 1
    while i < nyears*4:
        floatpmtdates.append(floatpmtdates[i-1]+relativedelta(months=+3))
        i+=1
#    fixedpmtdates = bdayconvert(start,end,fixedpmtdates) 
    floatpmtdates = bdayconvert(start,end,floatpmtdates)
    return floatpmtdates

In [212]:
def price(path1, path2, start,end,notional):
    notional = notional
    ldata = pd.read_csv(path1)
    odata = pd.read_csv(path2)
    #fixedpmtdates = pmtdates(start,end)[0]
    floatpmtdates = pmtdates(start,end)
    #getting the days for payment period
    sdate = datetime.datetime.strptime(start, '%Y-%m-%d')
    ldata['Dates'] = (floatpmtdates[0]-sdate).days
    i = 1
    while i < len(ldata['Dates']):
        ldata['Dates'][i] = (floatpmtdates[i]-floatpmtdates[i-1]).days
        i+=1
    #calculating interest
    ldata['Pmts'] = (notional*ldata['Forward Rate']/100)*ldata['Dates']/360
    #calculating the zero
    odata['Zerodf'] = (1/(1+odata['Forward Rate']/100))
    #calculating discount factors
    odata['DF'] = odata['Zerodf']
    i=1
    while i < len(odata['DF']):
        odata['DF'][i] = odata['Zerodf'][i]*odata['DF'][i-1]
        i+=1
    #pv floatingpmts
    ldata["PVpmts"]=1
    for i in range(len(ldata["PVpmts"])):
        ldata["PVpmts"][i] = ldata["Pmts"][i]*odata["DF"][i]

    #calculating the mid
    be = ("B/E: "+str(sum(ldata["PVpmts"])/sum(notional*(180/360)*odata['DF'])))
    dv = ("DV01: "+str(round(sum(notional*(180/360)*odata['DF'])*.0001,5)))
    return be, dv

In [213]:
def myClick():
    loading = Label(root, text= "Loading...")
    loading.grid(row=5, column =2)
    belabel = Label(root, text = price(str(p1.get()),str(p2.get()),str(s.get()),str(e.get()),float(n.get())))
    belabel.grid(row= 5, column = 2)

In [205]:
start = "2020-10-29"
end   = "2030-10-29"
notional = 100000000 #100 million
path1 = 'C:\\Users\\ECEI\\My Documents\\testdata.csv'
path2 = 'C:\\Users\\ECEI\\My Documents\\testois.csv'

In [214]:
root = Tk()

In [215]:
#creating label
mylabel = Label(root, text = "Swap Pricer (dealer conventions)")
mylabel2 = Label(root,text = "Input terms below")
slabel = Label(root,text="Start")
elabel = Label(root,text="end")
nlabel = Label(root,text="notional")
p1label= Label(root,text="P1")
p2label= Label(root,text="P2")
s  = Entry(root)
e  = Entry(root)
n  = Entry(root)
p1 = Entry(root)
p2 = Entry(root)
myButton = Button(root, text="Calculate", command = myClick)
mylabel.grid(row=0, column = 1)
mylabel2.grid(row = 1, column =1)
s.grid(row =2, column = 1)
e.grid(row=3, column = 1)
n.grid(row=4, column = 1)
p1.grid(row=2, column = 3)
p2.grid(row=3, column= 3)
slabel.grid(row=2,column=0)
elabel.grid(row =3, column =0)
nlabel.grid(row=4,column=0)
p1label.grid(row=2,column=2)
p2label.grid(row=3,column=2)
myButton.grid(row = 5, column = 1)

In [216]:
#event loop = program is always looping, therefore it knows where ur cursor is, if u click button
root.mainloop()

C:\Users\ECEI\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\ECEI\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ECEI\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [142]:
class swap:
    def __init__(self,start,end,notional):
        self.start = start
        self.end = end
        self.notional = notional
    def set_start(self,start):
        self.start = start
    def set_end(self,end):
        self.end = end
    def set_not(self,notional):
        self.notional = notional

In [143]:
s= swap("2020-10-29","2030-10-29",10000000)